US red meat and poultry forecast reports are available via the link below:
https://usda.library.cornell.edu/concern/publications/g445cd121?locale=en

Two document types are provided: 
1) pdf
2) xlsx spreadsheet

The Poultry forecast report table is found on the second last page of the PDF report.

Note:
- The data is structured in a nested tabular format making it somewhat more complicated to parse and extract the useful values from.
- Forecasts are provided in quarterly and annual frequency, with historic (past) and future forecast values provided in the same table
- Can integrate this into our pipeline by:
    1) extract the full historic / future forecast values provided in each report
    2) Resample data at daily frequency using using spline interpolation

The primary data of interest is:
    -'Eggs, Grade A large, New York, volume buyers, cents/dozen' found within the Market Prices section
- Secondary data of interest are: 
    -'Table eggs, million dozen' found within the Production section
    -'Eggs, number' found within the Per capita disappearance, retail pounds 1/

Since the price forecasts are specific to the New York market, we will need to translate values as best as possible to the National Market by:
1) Normalize prices by the current new york market egg price
    - This can be done using the spline interpolated daily frequency value, or alternatively by accessing the 'Daily New York Shell Egg' reports
    found at: 
        - https://usda.library.cornell.edu/concern/publications/xw42n7934?locale=en
2) Multiply the relative price movement price forecast series by the current National Egg price value
This uses New York market foreasts as a proxy for national average egg prices which will have its limitations, but it should hopefully 
still provide some informative predictive gain to our models

Provided in the cell below is a dictionary with most of the historic report url links provided...

You will find that the oldest reports may be structured slightly differently and that data is only available in pdf format...
we may therefore want to have the document parsing be restricted to using the pdf format documents, but this is not strictly required.

In [ ]:
usda_forecast_reports = [
    {'release_date': 'Jan 16, 2025',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6395z2696/sj13bz31x/LDP-M-367.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6395z2696/0c485d13p/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Dec 16, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8910mn86m/xp68nc45x/LDP-M-366.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8910mn86m/vq281h34q/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Nov 15, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6395z1622/st74fh665/LDP-M-365.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6395z1622/7m01df36r/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Oct 18, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/df65x253q/cz30rm50b/LDP-M-364.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/df65x253q/mk61tb46c/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Sep 18, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/js958775n/kp78j918v/LDP-M-363.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/js958775n/fx71cd287/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Aug 16, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/sx61gc32c/h415r234m/LDP-M-362.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/sx61gc32c/ks65k4716/Dairy_Forecasts.xlsx'
    },
    {'release_date': 'Jul 18, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/2j62tw21b/8g84pb98h/LDP-M-361.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/2j62tw21b/3j334t202/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Jun 18, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/9s162w66n/q811n9956/LDP-M-360.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/9s162w66n/9593wk21r/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'May 16, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/v118t4798/6d571n90s/LDP-M-359.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/v118t4798/ww72d102h/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Apr 17, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/1z40ng112/p5549g600/LDP-M-358.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/1z40ng112/9w033q09v/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Mar 14, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/9z904n56n/w0894039v/LDP-M-357.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/9z904n56n/1j92hw79g/Dairy_Forecasts.xlsx'
    },
    {'release_date': 'Feb 14, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8910mf43k/wm119b841/LDP-M-356.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8910mf43k/wh248c787/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Jan 19, 2024',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/kw52kw773/9593wg150/LDP-M-355.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/kw52kw773/mk61t476v/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Dec 14, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/k0699t751/f7625037b/LDP-M-354.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/k0699t751/xs55p1036/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Nov 16, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/tm70pd869/2514q475k/LDP-M-353.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/tm70pd869/9593wd815/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Oct 18, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/gb19gq76m/db78vx587/LDP-M-352.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/gb19gq76m/mk61t291j/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Sep 18, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/w3764s18j/5h73rd03f/LDP-M-351.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/w3764s18j/2r36wh22b/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Aug 17, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/sq87d961p/fx71c409c/LDP-M-350.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/sq87d961p/4b29cq200/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Jul 18, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/v979wk216/3x818391c/LDP-M-349.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/v979wk216/8910m890x/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Jun 15, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/zg64w2506/05743681k/LDP-M-348.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/zg64w2506/4q77h6003/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'May 18, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/gm80k9612/tt44r265g/LDP-M-347.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/gm80k9612/9s162m261/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Apr 17, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/h415qq287/gm80k899s/LDP-M-346.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/h415qq287/nk323t33f/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Mar 14, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6d571b47n/gm80k8225/LDP-M-345.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6d571b47n/w0893q65n/U.S__red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Feb 14, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/vt151w880/p5549547k/LDP-M-344.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/vt151w880/6w925p61x/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Jan 19, 2023',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/gx41nx08z/6w925p448/LDP-M-343.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/gx41nx08z/1257c535p/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Dec 15, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/c821hw77v/c247g3903/LDP-M-342.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/c821hw77v/xs55nq192/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Nov 16, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cz30r401h/7d27b369h/LDP-M-341.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cz30r401h/c534gz22t/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Oct 18, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cf95kk93b/fx71bx08p/LDP-M-340.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cf95kk93b/pz50j547t/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Sep 16, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/9w033934q/5138kq43q/LDP-M-339.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/9w033934q/8s45rj444/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Aug 18, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qv33t464p/vt151s20q/LDP-M-338.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qv33t464p/1544cx501/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Jul 18, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/f7624k78x/rn302841m/LDP-M-337.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/f7624k78x/b8516w637/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Jun 16, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7p88dp28f/pn89fc50s/LDP-M-336.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7p88dp28f/mc87qw404/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'May 18, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/1g05gj33z/wh247x14t/LDP-M-335.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/1g05gj33z/gh93j4348/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Apr 14, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/sx61fr227/6q183r28m/LDP-M-334.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/sx61fr227/0k226h40r/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Mar 15, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/sn00c253f/4j03f258q/LDP-M-333.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/sn00c253f/f1882q27j/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Feb 15, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cr56p351s/cf95kd48g/LDP-M-332.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cr56p351s/k0699b32b/U.S._red_meat_and_poultry_forecasts.xlsx'
    },
    {'release_date': 'Jan 19, 2022',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/2227nr83z/4b29c877f/LDP-M-331.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/2227nr83z/q524kq976/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Dec 15, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/fn108001h/hd76t186t/LDP-M-330.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/fn108001h/vx022g17d/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Nov 16, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/4742b8914/dz011q64j/LDP-M-329.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/4742b8914/w0893c29q/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Oct 18, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6h441s74x/f7624c233/LDP-M-328.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6h441s74x/bz60dv934/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Sep 16, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cn69n276k/wd376v872/LDP-M-327.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cn69n276k/44559d01x/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Aug 18, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/gx41ng932/2b88rb78f/LDP-M-326.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/gx41ng932/p5548q811/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Jul 16, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/5m60rq15s/sq87cr38d/LDP-M-324.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/5m60rq15s/gt54mj599/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Jun 16, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/0v838x237/f4753d50b/LDP-M-324.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/0v838x237/td96kz756/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'May 18, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/4j03dt75r/bc387f479/LDP-M-323.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/4j03dt75r/2r36vt80x/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Apr 15, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qj72q2880/4q77gk83z/LDP-M-322.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qj72q2880/bk129641x/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Mar 15, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/fj236w85w/b2774p82g/LDP-M-321.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/fj236w85w/qn59qz448/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Feb 18, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cc08j8026/vq280f939/LDP-M-320.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cc08j8026/h128p809g/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Jan 19, 2021',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7s75f536w/j0990467x/LDP-M-319.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7s75f536w/xw42p270z/US-red-meat-and-poultry-forecasts.xlsx'
    },
    {'release_date': 'Dec 16, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/3t946g36j/k643bs51s/LDP-M-318.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/3t946g36j/h989rv27k/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Nov 17, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7s75f406z/g732f239d/LDP-M-317.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7s75f406z/dr26zn983/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Oct 16, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6969zq68h/2n49tr940/LDP-M-316.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6969zq68h/hq37wc47w/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Sep 17, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qj72px324/ht24x770q/LDP-M-315.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qj72px324/z890sh685/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Aug 18, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/pv63gn53t/2j62st15m/LDP-M-314.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/pv63gn53t/47429x85r/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Jul 16, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/ng452570s/7p88d445s/LDP-M-313.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/ng452570s/6969zn330/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Jun 17, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/fx71b794m/k3569q949/LDP-M-312.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/fx71b794m/zc77tb45z/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'May 18, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/mc87q950v/1z40md459/LDP-M-311.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/mc87q950v/hx11z1867/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Apr 15, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/5d86pj126/bz60df867/LDP-M-310.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/5d86pj126/02871f53t/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Mar 16, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/0z709d96h/zs25xt64m/LDP-M-309.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/0z709d96h/8p58px629/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Feb 18, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/k3569n13p/t435gw59k/LDP-M-308.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/k3569n13p/4b29bq612/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Jan 16, 2020',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qv33sc91v/pv63gg17x/LDP-M-307.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qv33sc91v/j098zt64x/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Dec 16, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/pg15bw198/hq37w4023/LDP-M-306.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/pg15bw198/mk61rz15r/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Nov 15, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/th83mc79n/c534g352c/LPDM-Outlook-November-2019.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/th83mc79n/v118rv224/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Oct 17, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/pr76fh28s/d504s037x/LDP-M-304.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/pr76fh28s/kd17d733j/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Sep 18, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/v405sq02p/rv043659g/ldp-m-303.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/v405sq02p/nc5812050/us-red-meat-and-poultry-forecasts__12_.xls'
    },
    {'release_date': 'Aug 16, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/vm40z409n/4m90f755k/LPD-M-302.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/vm40z409n/h415pn89c/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Jul 17, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qv33s753m/9019sd30t/LDPM301.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qv33s753m/ht24ww17z/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Jun 17, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/vm40z269p/6108vn30b/LPDM300.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/vm40z269p/r494vv973/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'May 16, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7d2792872/4x51ht669/LDPM299.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7d2792872/6395wg67s/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Apr 15, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/5q47rw77s/zg64tv126/LPDM298.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/5q47rw77s/gt54kw464/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Mar 14, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/q237j0261/0z709366x/LPDM297.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/q237j0261/w3763f057/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Feb 14, 2019',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7d279088x/h702qd44f/ldpm296.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7d279088x/pg15bn224/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Dec 17, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cr56n493b/1j92gc85b/LPDM294.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cr56n493b/bc386p57t/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Nov 15, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/0z709072f/v405sd67n/LPDM293.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/0z709072f/vd66w323j/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Oct 17, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7s75dg669/cc08hj96z/LPDM292.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/7s75dg669/bg257j62q/US-red-meat-and-poultry-forecasts.xls'
    },
    {'release_date': 'Sep 18, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/2r36v288k/zc77sv01t/LDP-M-09-18-2018.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/2r36v288k/mw22v9401/LDP-M-09-18-2018.xls'
    },
    {'release_date': 'Aug 16, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/zp38wf18c/0z708z04x/LDP-M-08-16-2018.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/zp38wf18c/r494vm54v/LDP-M-08-16-2018.xls'
    },
    {'release_date': 'Jul 18, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cf95jc74q/n870zs36w/LDP-M-07-18-2018.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cf95jc74q/8623j028r/LDP-M-07-18-2018.xls'
    },
    {'release_date': 'Jun 18, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/bv73c1964/0z708z03n/LDP-M-06-18-2018.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/bv73c1964/dn39x310s/LDP-M-06-18-2018.xls'
    },
    {'release_date': 'May 16, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/nk322f993/7p88cj28f/LDP-M-05-16-2018.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/nk322f993/db78td65c/LDP-M-05-16-2018.xls'
    },
    {'release_date': 'Apr 16, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8049g676x/df65v9500/LDP-M-04-16-2018.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8049g676x/6t053h61w/LDP-M-04-16-2018.xls'
    },
    {'release_date': 'Mar 14, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/h989r478q/6682x566h/LDP-M-03-14-2018.pdf',
     'xlsx_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/h989r478q/zk51vj215/LDP-M-03-14-2018.xls'
    },
    {'release_date': 'Feb 14, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/pv63g1886/hm50tt41m/LDP-M-02-14-2018.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Jan 19, 2018',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/r781wh68f/d504rm897/LDP-M-01-19-2018.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Dec 18, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/bn999823b/7p88cj275/LDP-M-12-18-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Nov 16, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/zk51vj20w/kk91fn02j/LDP-M-11-16-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Oct 18, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6q182m60c/w66345134/LDP-M-10-18-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Sep 18, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/2r36v030r/s4655j16f/LDP-M-09-18-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Aug 16, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/h989r477f/9w0324451/LDP-M-08-16-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Jul 18, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cc08hh24h/r494vm53k/LDP-M-07-18-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Jun 15, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/b8515p928/7m01bn32j/LDP-M-06-15-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'May 16, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/05741t22s/zg64tn46s/LDP-M-05-16-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Apr 17, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/xs55md629/79407z742/LDP-M-04-17-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Mar 15, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/4x51hk57w/k930bz47x/LDP-M-03-15-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Feb 15, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/2b88qf16w/00000158m/LDP-M-02-15-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Jan 19, 2017',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cv43nz40z/r494vm529/LDP-M-01-19-2017.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Dec 15, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/b5644t332/0g354g75m/LDP-M-12-15-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Nov 16, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/x920fz35w/6q182m61n/LDP-M-11-16-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Oct 18, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/9w032444r/ws859h41w/LDP-M-10-18-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Sep 16, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/qj72p871w/bn999824m/LDP-M-09-16-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Aug 18, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/k643b264h/6t053h60m/LDP-M-08-18-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Jul 18, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/fb4949990/cj82k899t/LDP-M-07-18-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Jun 16, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/v692t7818/j67315590/LDP-M-06-16-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'May 16, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/ns064752z/qf85nd00w/LDP-M-05-16-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Apr 18, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/g445cf67t/8w32r7279/LDP-M-04-18-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Mar 15, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6682x5657/6t053h59v/LDP-M-03-15-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Feb 16, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/mk61rj272/z890rv860/LDP-M-02-16-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Jan 19, 2016',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/jm214q803/k643b265s/LDP-M-01-19-2016.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Dec 15, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/1j92g914z/2801ph939/LDP-M-12-15-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Nov 17, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/ns064751p/x920fz365/LDP-M-11-17-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Oct 16, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/ms35t998k/vm40xt30v/LDP-M-10-16-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Sep 17, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/zw12z705s/mc87pr86r/LDP-M-09-17-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Aug 18, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/zs25xb162/4m90dx09f/LDP-M-08-18-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Jul 16, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/cf95jc73f/qf85nc99b/LDP-M-07-16-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Jun 16, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/gf06g4195/rj4305983/LDP-M-06-16-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'May 18, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/6q182m59m/1g05fd194/LDP-M-05-18-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Apr 15, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/zk51vj194/08612p99f/LDP-M-04-15-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Mar 16, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/vx021g754/p2676x044/LDP-M-03-16-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Feb 17, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8k71nj74p/cv43nz417/LDP-M-02-17-2015.pdf',
     'xlsx_url': ''
    },
    {'release_date': 'Jan 16, 2015',
     'pdf_url': 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/h128ng45d/0k225c69q/LDP-M-01-16-2015.pdf',
     'xlsx_url': ''
    },
]

The following cells below include some initial testing I've done aiming to parse the xlsx reports... 

In [ ]:
usda_forecasts_url = 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8910mn86m/vq281h34q/U.S._red_meat_and_poultry_forecasts.xlsx'

usda_df = pd.read_excel(usda_forecasts_url, skiprows=2)

usda_df

In [ ]:
from openpyxl import load_workbook
import csv

# Define the URL and local filename
url = 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8910mn86m/vq281h34q/U.S._red_meat_and_poultry_forecasts.xlsx'
local_file = 'temp_file.xlsx'

# Download the file
import requests
response = requests.get(url)
with open(local_file, 'wb') as file:
    file.write(response.content)

# Load the workbook
workbook = load_workbook(local_file)

# Select the first sheet (adjust as needed)
sheet = workbook.active

# Define the CSV output file
csv_file = 'output.csv'

# Open CSV file for writing
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write rows from the Excel sheet to the CSV
    for row in sheet.iter_rows(values_only=True):
        writer.writerow(row)

print(f"Data extracted and saved to {csv_file}")


In [ ]:
from openpyxl import load_workbook
import requests

# Define the URL and local filename
url = 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8910mn86m/vq281h34q/U.S._red_meat_and_poultry_forecasts.xlsx'
local_file = 'temp_file.xlsx'

# Download the file
response = requests.get(url)
with open(local_file, 'wb') as file:
    file.write(response.content)

# Load the workbook
workbook = load_workbook(local_file)

# Extract text content
text_output = []

for sheet in workbook.sheetnames:  # Iterate through all sheets
    text_output.append(f"--- Sheet: {sheet} ---\n")
    worksheet = workbook[sheet]
    for row in worksheet.iter_rows(values_only=True):  # Iterate through all rows
        line = "\t".join([str(cell) if cell is not None else "" for cell in row])
        text_output.append(line)
    text_output.append("\n")

# Join all text and save to a text file
output_file = 'output_text.txt'
with open(output_file, 'w', encoding='utf-8') as file:
    file.write("\n".join(text_output))

print(f"Data extracted as text and saved to {output_file}")

print(''.join(text_output))

In [ ]:
from openpyxl import load_workbook
import json

# Define the URL and local filename
url = 'https://downloads.usda.library.cornell.edu/usda-esmis/files/g445cd121/8910mn86m/vq281h34q/U.S._red_meat_and_poultry_forecasts.xlsx'
local_file = 'temp_file.xlsx'

# Download the file
import requests
response = requests.get(url)
with open(local_file, 'wb') as file:
    file.write(response.content)

# Load the workbook
workbook = load_workbook(local_file)

# Prepare structured data for LLM
data = {}

for sheet_name in workbook.sheetnames:  # Iterate through all sheets
    sheet = workbook[sheet_name]
    sheet_data = []
    for row in sheet.iter_rows(values_only=True):  # Extract row data
        sheet_data.append([cell if cell is not None else "" for cell in row])
    data[sheet_name] = sheet_data

# Convert to JSON or other formats
output_file = 'data_for_llm.json'
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4)

print(f"Data extracted and saved to {output_file}")

print(json.dumps(data, indent=4))
